In [1]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:2242/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def get_llama_response(prompt, temperature = 0.1, max_tokens = 128):
    completion = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature = temperature,
        # max_tokens = max_tokens
    )

    return completion.choices[0].message.content

In [2]:
get_llama_response("৫ জন বাংলাদেশি বিখ্যাত কবির নাম বল?")

'বাংলাদেশে ৫ জন উল্লেখযোগ্য কবির নাম নিম্নরূপ:\n\n1. কাজী নজরুল ইসলাম\n2. আল্লামা ইকবাল\n3. মনুজুর রহমান\n4. শহীদ কাবির\n5. সোনার্জা'

In [3]:
import pandas as pd
from datasets import load_dataset

# Load the Bengali subset of the dataset
dataset = load_dataset('mteb/IndicSentiment', 'bn')

# Convert to pandas dataframe
df = dataset['train'].to_pandas()  # Assuming you want the training split

# Display the dataframe
df.head()

README.md:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

bn.jsonl.gz:   0%|          | 0.00/239k [00:00<?, ?B/s]

bn.jsonl.gz:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/156 [00:00<?, ? examples/s]

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,এই বোটের সাউন্ডবারটি এখনও সব স্পিকারের জন্য তা...,bn
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,মাইক ও মাইক্রো এসডি কার্ড স্লটসহ ভাঁজযোগ্য ধরন...,bn
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,24 ঘণ্টার জন্য গল্পের যে বৈশিষ্ট্যটি ডিফল্টভাব...,bn
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে ...",bn
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,দেখতে অনেক বড় এবং দক্ষ। কিন্তু যেহেতু সামনে কো...,bn


In [4]:
zero_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

"""

In [5]:
three_shot = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

### অনুভূতি:
পজিটিভ

### আপনার কাজ

"""

In [6]:
chain_of_thought = """আপনাকে একটি পণ্যের রিভিউ দেওয়া হয়েছে। আপনাকে রিভিউটির অনুভূতি নির্ধারণ করতে হবে, যা পজিটিভ বা নেগেটিভ হতে পারে। রিভিউটির অনুভূতি নির্ধারণ করতে গিয়ে নিম্নলিখিত ধাপগুলি অনুসরণ করুন:

### ধাপ ১: রিভিউ পড়া
রিভিউটি পড়ুন এবং এতে কী বলা হয়েছে তা বোঝার চেষ্টা করুন।

### ধাপ ২: ইতিবাচক বা নেতিবাচক শব্দ চিহ্নিত করা
রিভিউতে ব্যবহৃত ইতিবাচক (ভালো, সন্তুষ্ট, দুর্দান্ত) বা নেতিবাচক (খারাপ, বাজে, নিম্নমানের) শব্দগুলি চিহ্নিত করুন।

### ধাপ ৩: অনুভূতি নির্ধারণ
রিভিউতে ইতিবাচক শব্দ বেশি হলে পজিটিভ এবং নেতিবাচক শব্দ বেশি হলে নেগেটিভ অনুভূতি নির্ধারণ করুন।

### উদাহরণ ১
### রিভিউ:
দরগুলি প্রতিযোগিতামূলক, প্রায় সবসময় বাজারে সেরা।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: প্রতিযোগিতামূলক, সেরা
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### উদাহরণ ২
### রিভিউ:
অন কল কানেক্টিভিটি অনেক সময় খুব কম হয়।

#### চিন্তাধারা:
- নেতিবাচক শব্দ: অনেক সময় খুব কম
- অনুভূতি: নেগেটিভ

### অনুভূতি:
নেগেটিভ

### উদাহরণ ৩
### রিভিউ:
হুক এবং লুপ ডিজাইন সহজেই সেট আপ এবং বহন করা যায়।

#### চিন্তাধারা:
- ইতিবাচক শব্দ: সহজেই সেট আপ এবং বহন করা যায়
- অনুভূতি: পজিটিভ

### অনুভূতি:
পজিটিভ

### আপনার কাজ


"""

In [7]:
category_translation = {
    'Positive': 'পজিটিভ',
    'Negative': 'নেগেটিভ',
}

In [8]:
path_root = 'exps/'

In [9]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_zs_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = zero_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root + 'llama_zs_sentiment_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_zs_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [10]:
zs_results = pd.read_parquet(path_root + 'llama_zs_sentiment_results.parquet')
zs_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,অনুভূতি: পজিটিভ\n\nএই রিভিউটিতে বলা হয়েছে যে ...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,এই পণ্যের রিভিউটির অনুভূতি প্রকৃতপক্ষে নেগেটিভ...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,রিভিউটি পজিটিভ নয়। এতে রিমোট কন্ট্রোল সিস্টেম...
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,রিভিউটি থেকে অনুমান করা হয় যে এটি সিনেমাটির প...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,রিভিউটি থেকে অনুভূতি হলো নেগেটিভ। রিভায়োয় মক...


In [11]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_ts_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = three_shot + " ###\nরিভিউ: " + review + "\n### অনুভূতি:"

    try:
        df.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root + 'llama_ts_sentiment_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_ts_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [12]:
ts_results = pd.read_parquet(path_root + 'llama_ts_sentiment_results.parquet')
ts_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,### উত্তর:\n### অনুভূতি:\nপজিটিভ\n\n### যুক্তি...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,রিভিউ থেকে দেখায় যে আইকলের টাওয়ার স্পিকার সেট...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,অনুভূতি: নেগেটিভ\n\nকারণ এখানে পণ্য (হোটেল) সম...
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,### অনুভূতি:\nপজিটিভ\n\nযেহেতু রিভিউতে বলা হয়...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,আমি রিভিউটিতে বিবৃতিটিকে পজিটিভ বা নেগেটিভ হিস...


In [13]:
import time
from tqdm.notebook import tqdm

# Assuming df is the original DataFrame and category_translation is defined
df['predicted_category'] = None

# Try to load the progress from the Parquet file
try:
    df_progress = pd.read_parquet(path_root + 'llama_cot_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df.copy()
    start_index = 0


for i, row in tqdm(df.iloc[start_index:].iterrows(), initial=start_index, total=len(df) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_bn = category_translation.get(label, label)  # Use the translated label or fallback to the original

    prompt_input = chain_of_thought + "\n### রিভিউ: " + review

    try:
        df.at[i, 'predicted_category'] = get_llama_response(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    if (i + 1) % 100 == 0:
        df.to_parquet(path_root +'llama_cot_sentiment_results.parquet')

# Save final progress
df.to_parquet(path_root + 'llama_cot_sentiment_results.parquet')

  0%|          | 0/156 [00:00<?, ?it/s]

In [14]:
cot_results = pd.read_parquet(path_root + 'llama_cot_sentiment_results.parquet')
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
151,Home,Appliances,Home theater,Home theater systems,iKall,"Bluetooth, USB &HDMI, Dolby, voice control, sp...",Dolby output,IKall has now launched a new home theater syst...,Positive,আইকল এখন ডলবি আউটপুট সহ একটি নতুন হোম থিয়েটার ...,bn,আমার কাজ হল প্রদত্ত রিভিউটির অনুভূতি নির্ধারণ ...
152,Home,Appliances,Home theater,Tower speakers,iKall,"speaker connectivity, speaker feature, wattage...",Wattage,iKall is giving two 500 Watts speakers in its ...,Negative,আইকল তাদের টাওয়ার স্পিকার সেটে 500 ওয়াটের দুটি...,bn,### রিভিউ: আইকল তাদের টাওয়ার স্পিকার সেটে 500 ...
153,Health/Wellness,Living,Stay/ Experience,Resorts,Vedic Village Spa Resort (Kolkata),"Swimming pool, fitness center, parking, wifi, ...","Wifi, air conditioning, family friendly","The Resort doesn't have wifi, so you have to b...",Negative,"রিসোর্টে ওয়াইফাই নেই, তাই আপনাকে আপনার ফোন ইন...",bn,### এই রিভিউটি মূল্যায়ন করা হচ্ছে\n\n\n#### চ...
154,Entertainment,Movies,Genres,Tragedy,The Tunnel,"serious, storyline, performances, emotional, m...",Peformances and Moving,"Well directed, & acted, & excellent cinematogr...",Positive,"ভালো পরিচালনা এবং অভিনয়, একটি অসাধারণ সিনেমাটো...",bn,### ধাপ ১: রিভিউ পড়া\nরিভিউটি পড়ুন এবং এতে ক...
155,Fashion,Beauty,Fragrance,Deodorants for women,Dove Advanced Care Antiperspirant Deodorant Se...,"alcohol free, skin whitening, odour control, l...",Long lasting,Its a terrible product!! Works only for a few ...,Negative,এটা একটা খারাপ প্রোডাক্ট!! মাত্র কয়েক ঘণ্টা কা...,bn,### অনুভূতি নির্ধারণ করতে আমরা নিম্নলিখিত ধাপগ...


In [15]:
def clean_predicted_category(text, category_translation):
    # Check if the text is None
    if text is None:
        return ""

    # Check if any category is directly present in the prediction string
    for category in category_translation.values():
        if category in text:
            return category

    # If no direct match, extract the text within ** ** and remove spaces
    import re
    match = re.search(r'\*\*([^*]*)\*\*', text)
    if match:
        cleaned_text = match.group(1).replace(" ", "")
        # Check if cleaned text matches any category in Bangla
        for category in category_translation.values():
            if cleaned_text == category:
                return category

    # If no match found, return None
    return ""

In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

import numpy as np

def update_cleaned_predicted_category(row):
    if pd.isna(row['cleaned_predicted_category']) or row['cleaned_predicted_category'] == '':
        if row['category_bn'] == 'পজিটিভ':
            return 'নেগেটিভ'
        elif row['category_bn'] == 'নেগেটিভ':
            return 'পজিটিভ'
    return row['cleaned_predicted_category']

def evaluate_classification(df):
    df['category'] = df['LABEL']
    # Map English category to Bangla
    df['category_bn'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Update the cleaned predicted categories
    for index, row in df.iterrows():
        old_value = row['cleaned_predicted_category']
        if pd.isna(old_value) or old_value == '':
            if row['category_bn'] == 'পজিটিভ':
                new_value = 'নেগেটিভ'
            elif row['category_bn'] == 'নেগেটিভ':
                new_value = 'পজিটিভ'
            df.at[index, 'cleaned_predicted_category'] = new_value

    unique_labels = set(df['category_bn'].unique()).union(set(df['cleaned_predicted_category'].unique()))
    if len(unique_labels) > 2:
        raise ValueError(f"Expected binary classification but found multiple labels: {unique_labels}")

    # Calculate metrics
    y_true = df['category_bn']
    y_pred = df['cleaned_predicted_category']


    # Ensure the labels are binary
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    if len(lb.classes_) != 2:
        raise ValueError(f"Expected binary classification but found these classes: {lb.classes_}")

    pos_label = lb.classes_[1]  # The positive label

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics

In [18]:
metrics = evaluate_classification(zs_results)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.7692
precision: 0.7532
recall: 0.7733
f1_score: 0.7632
auc_roc: 0.7694
log_loss: 8.3178


In [19]:
metrics = evaluate_classification(ts_results)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.8526
precision: 0.7889
recall: 0.9467
f1_score: 0.8606
auc_roc: 0.8560
log_loss: 5.3141


In [20]:
metrics = evaluate_classification(cot_results)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.7115
precision: 0.6316
recall: 0.9600
f1_score: 0.7619
auc_roc: 0.7207
log_loss: 10.3972
